In [1]:
# SEP769 - July 27, 2021
# Amir Kamaleddine, Ernani Fantinatti, Mohammed Ibraheem
# LSTM model for classifying normal and imbalanced motors as part of the MAFAULDA motor fault dataset
# If training, we suggest you use GPU acceleration in Colab
# https://github.com/efantinatti/MAFAULDA_SEP769

In [2]:
# Load in tensorboard extension to view results interactively
%load_ext tensorboard


In [3]:
# Load relevant libraries
import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, SimpleRNN, LSTM
import matplotlib.pyplot as plt
import numpy as np
import glob
import datetime


In [4]:
# For Google Colab only
# Pulling processed data from public folder
# Credit to this stack overflow forum: https://stackoverflow.com/questions/38511444/python-download-files-from-google-drive-using-url
#import gdown

#url = 'https://drive.google.com/uc?id=1otE_HKfM24r9Hp1-e5xodSkgve1o3DO5' 
#output = 'jul27data.zip'
#gdown.download(url, output, quiet=False)

#!unzip /content/jul27data.zip
#!rm -rf sample_data


In [5]:
# Decide whether to load model from a .h5 file or train a new one
saved_model = 0


In [6]:
#Delete tensorboard logs from previous runs
#If using Collab or Unix system use !rm, otherwise
#if using windows skip this
#!rm -rf ./logs/


In [7]:
# Loads train/test data from .txt files to np arrays 
# Ensure you choose the correct file directory

dir = 'C:/Users/KOMPUTER/Documents/Jupyter Notebooks/MAFAULDA_SEP769-main/Models/content/jul27data/'

train_reload = np.loadtxt(dir + 'train_data.txt')
test_reload = np.loadtxt(dir + 'test_data.txt')

print("Loaded data files...")


Loaded data files...


In [8]:
# Shaping the training/testing data
# Shapes the data into an nx50x8 array
# Each sequence is 50 timesteps (1 ms of data collection)

train_data = np.reshape(train_reload,(-1,50, 8))
test_data = np.reshape(test_reload,(-1,50, 8))


In [9]:
# Loads labels corresponding to training/test data
train_label = np.loadtxt(dir + 'train_label.txt').astype(int)
test_label = np.loadtxt(dir + 'test_label.txt').astype(int)

# Organized data is labeled per 500 timesteps
# To align the number of labels with number of sequences, need to correct number of labels
# These lines correct the number of labels
A = sum(train_label == 1)
B = sum(test_label == 1)
train_label = np.zeros((train_label.shape-A)*10)
train_label = np.append(train_label,np.ones(A*10))
test_label = np.zeros((test_label.shape-B)*10)
test_label = np.append(test_label,np.ones(B*10))


In [16]:
# Build the model
# Sequential model
model = Sequential()

# LSTM layers
model.add(LSTM(24, input_shape = (50,8), activation = 'tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(12, activation = 'tanh', recurrent_activation='sigmoid', return_sequences=True))
model.add(LSTM(4, activation = 'tanh', recurrent_activation='sigmoid'))

# Output layer
model.add(Dense(1, activation='sigmoid'))

# Compile model
# Binary crossentropy for a binary classifier
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


In [17]:
ld = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
callTensorboard = tf.keras.callbacks.TensorBoard(log_dir=ld, histogram_freq=1)


In [18]:
if saved_model == 0:
    # Train model
    # Note that validation data is NOT used to train model at all
    history = model.fit(train_data, train_label, validation_data=(test_data, test_label), callbacks=[callTensorboard], epochs=50, batch_size=50)
    model.save('C:/Users/KOMPUTER/Documents/Jupyter Notebooks/MAFAULDA_SEP769-main/Models/LSTM.h5')    
else:
    # Load model from .h5 file
    model = load_model('C:/Users/KOMPUTER/Documents/Jupyter Notebooks/MAFAULDA_SEP769-main/Models/LSTM.h5')


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [ ]:
# Evaluate model performance with test set
test_accuracy = model.evaluate(test_data, test_label)


In [ ]:
# Plot model accuracy over epoch if trained
if saved_model == 0:
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper right')
    plt.show()
    

In [ ]:
if saved_model == 0:
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
%tensorboard --logdir logs/fit
